In [35]:
import json
import pickle
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
import torch.nn.functional as F
import spacy
import numpy as np

import random
import math
import time

from torchtext.data import TabularDataset

In [36]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [1]:
!wget https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl

--2020-12-30 05:54:55--  https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.47.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.47.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785890 (3.6M) [binary/octet-stream]
Saving to: ‘train_rand_split.jsonl’

train_rand_split.js 100%[===================>]   3.61M  7.61MB/s    in 0.5s    

2020-12-30 05:54:56 (7.61 MB/s) - ‘train_rand_split.jsonl’ saved [3785890/3785890]



In [3]:
!wget https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl

--2020-12-30 05:55:01--  https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471653 (461K) [binary/octet-stream]
Saving to: ‘dev_rand_split.jsonl’

dev_rand_split.json 100%[===================>] 460.60K  1.75MB/s    in 0.3s    

2020-12-30 05:55:02 (1.75 MB/s) - ‘dev_rand_split.jsonl’ saved [471653/471653]



In [32]:
import json
import pandas as pd
import os

In [5]:
df = pd.read_json('/content/dev_rand_split.jsonl',lines= True)

In [6]:
df.head()

,answerKey,id,question
0,A,1afa02df02c908a558b4036e80242fac,"{'question_concept': 'revolving door', 'choice..."
1,A,a7ab086045575bb497933726e4e6ad28,"{'question_concept': 'people', 'choices': [{'l..."
2,B,b8c0a4703079cf661d7261a60a1bcbff,"{'question_concept': 'magazines', 'choices': [..."
3,A,e68fb2448fd74e402aae9982aa76e527,"{'question_concept': 'hamburger', 'choices': [..."
4,A,2435de612dd69f2012b9e40d6af4ce38,"{'question_concept': 'farmland', 'choices': [{..."


In [14]:
question = []
answer = []
for i in range(df.shape[0]):
  question.append(df.iloc[i].question['stem'])
  choice = df.iloc[i].question['choices']
  for ii in range(len(choice)):
    if choice[ii]['label'] == df.iloc[i].answerKey:
      answer.append(choice[ii]['text'])





In [15]:
df['Question'] = question
df['Answer'] = answer

In [16]:
df.head()

,answerKey,id,question,Question,Answer
0,A,1afa02df02c908a558b4036e80242fac,"{'question_concept': 'revolving door', 'choice...",A revolving door is convenient for two directi...,bank
1,A,a7ab086045575bb497933726e4e6ad28,"{'question_concept': 'people', 'choices': [{'l...",What do people aim to do at work?,complete job
2,B,b8c0a4703079cf661d7261a60a1bcbff,"{'question_concept': 'magazines', 'choices': [...",Where would you find magazines along side many...,bookstore
3,A,e68fb2448fd74e402aae9982aa76e527,"{'question_concept': 'hamburger', 'choices': [...",Where are you likely to find a hamburger?,fast food restaurant
4,A,2435de612dd69f2012b9e40d6af4ce38,"{'question_concept': 'farmland', 'choices': [{...",James was looking for a good place to buy farm...,midwest


In [18]:
dev_df = df[['Question','Answer']]
dev_df.head()

,Question,Answer
0,A revolving door is convenient for two directi...,bank
1,What do people aim to do at work?,complete job
2,Where would you find magazines along side many...,bookstore
3,Where are you likely to find a hamburger?,fast food restaurant
4,James was looking for a good place to buy farm...,midwest


In [34]:
if not os.path.exists('preprocessed_valid.csv'):
    dev_df.to_csv('preprocessed_valid.csv', index = False)

In [25]:
train_df =  pd.read_json('/content/train_rand_split.jsonl',lines= True) 

In [26]:
train_df.head()

,answerKey,id,question
0,A,075e483d21c29a511267ef62bedc0461,"{'question_concept': 'punishing', 'choices': [..."
1,B,61fe6e879ff18686d7552425a36344c8,"{'question_concept': 'people', 'choices': [{'l..."
2,A,4c1cb0e95b99f72d55c068ba0255c54d,"{'question_concept': 'choker', 'choices': [{'l..."
3,D,02e821a3e53cb320790950aab4489e85,"{'question_concept': 'highway', 'choices': [{'..."
4,C,23505889b94e880c3e89cff4ba119860,"{'question_concept': 'fox', 'choices': [{'labe..."


In [27]:
question = []
answer = []
for i in range(train_df.shape[0]):
  question.append(train_df.iloc[i].question['stem'])
  choice = train_df.iloc[i].question['choices']
  for ii in range(len(choice)):
    if choice[ii]['label'] == train_df.iloc[i].answerKey:
      answer.append(choice[ii]['text'])

In [28]:
train_df['Question'] = question
train_df['Answer'] = answer

In [29]:
train_df = train_df[['Question','Answer']]
train_df.head()

,Question,Answer
0,The sanctions against the school were a punish...,ignore
1,Sammy wanted to go to where the people were. ...,populated areas
2,To locate a choker not located in a jewelry bo...,jewelry store
3,Google Maps and other highway and street GPS s...,atlas
4,"The fox walked from the city into the forest, ...",natural habitat


In [33]:
if not os.path.exists('preprocessed_train.csv'):
    train_df.to_csv('preprocessed_train.csv', index = False)

In [37]:
spacy_en = spacy.load('en')

In [38]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [39]:
QSTN = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANS = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [41]:
trainset = TabularDataset(path='preprocessed_train.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [42]:
test_data = TabularDataset(path='preprocessed_valid.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [43]:
train_data, valid_data = trainset.split(split_ratio = 0.7, random_state=random.getstate())  

In [44]:
vars(train_data.examples[0])

{'Answer': ['fifties'],
 'Question': ['jerry',
  'was',
  'too',
  'old',
  'for',
  'this',
  'shit',
  '.',
  ' ',
  'he',
  'was',
  'too',
  'square',
  ',',
  'too',
  '.',
  ' ',
  'he',
  'never',
  'did',
  'drugs',
  ',',
  'never',
  'drank',
  ',',
  'but',
  'he',
  'was',
  'in',
  'his',
  'what',
  '?']}

In [45]:
vars(valid_data.examples[0])

{'Answer': ['maternity', 'ward'],
 'Question': ['where',
  'would',
  'you',
  'find',
  'a',
  'waiting',
  'room',
  'with',
  'only',
  'some',
  'fathers',
  '?']}

In [46]:
QSTN.build_vocab(train_data, min_freq = 2)
ANS.build_vocab(train_data, min_freq = 2)

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [61]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort_key=lambda x: len(x.Question), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,)

In [49]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [50]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [51]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [52]:
INPUT_DIM = len(QSTN.vocab)
OUTPUT_DIM = len(ANS.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [53]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3725, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1454, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=1454, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,340,270 trainable parameters


In [55]:
optimizer = optim.Adam(model.parameters())

In [56]:
TRG_PAD_IDX = ANS.vocab.stoi[ANS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [57]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        # print('Training Done')
    return epoch_loss / len(iterator)

In [58]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            # print(src)
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [59]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [62]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 3.963 | Train PPL:  52.598
	 Val. Loss: 3.603 |  Val. PPL:  36.722
Epoch: 02 | Time: 0m 1s
	Train Loss: 3.851 | Train PPL:  47.037
	 Val. Loss: 3.596 |  Val. PPL:  36.469
Epoch: 03 | Time: 0m 1s
	Train Loss: 3.779 | Train PPL:  43.762
	 Val. Loss: 3.576 |  Val. PPL:  35.725
Epoch: 04 | Time: 0m 1s
	Train Loss: 3.758 | Train PPL:  42.868
	 Val. Loss: 3.596 |  Val. PPL:  36.462
Epoch: 05 | Time: 0m 1s
	Train Loss: 3.738 | Train PPL:  41.994
	 Val. Loss: 3.587 |  Val. PPL:  36.133
Epoch: 06 | Time: 0m 1s
	Train Loss: 3.714 | Train PPL:  41.036
	 Val. Loss: 3.583 |  Val. PPL:  35.975
Epoch: 07 | Time: 0m 1s
	Train Loss: 3.705 | Train PPL:  40.666
	 Val. Loss: 3.587 |  Val. PPL:  36.115
Epoch: 08 | Time: 0m 1s
	Train Loss: 3.694 | Train PPL:  40.216
	 Val. Loss: 3.591 |  Val. PPL:  36.261
Epoch: 09 | Time: 0m 1s
	Train Loss: 3.658 | Train PPL:  38.793
	 Val. Loss: 3.594 |  Val. PPL:  36.394
Epoch: 10 | Time: 0m 1s
	Train Loss: 3.680 | Train PPL:  39.653


In [63]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.556 | Test PPL:  35.012 |
